In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

import math
import datetime
import time

import requests
import json
from addict import Dict
import jmespath
import numpy as np
import pandas as pd
import time
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [ ]:
def do_request(func,**kwargs):
    response= func(**kwargs)
    if response.status_code==200:
        return response
    elif response.status_code==429:
        wait_after_seconds=int(response.headers["Retry-After"])
        print(f"429 Error\tWaiting seconds: {wait_after_seconds+1}")
        time.sleep(wait_after_seconds+1)
        return do_request(func,**kwargs)
    else:
        raise ValueError(f"{response.status_code}\t{response.text}")

In [ ]:
def get_ilceler(il_kodu):
    url = f"https://api.emlakjet.com/geolocation/location/{il_kodu}?fields=name,parentId,center,slug&includeChildren=true&includeRelated=false"

    payload = {}
    headers = {
      'authority': 'api.emlakjet.com',
      'pragma': 'no-cache',
      'cache-control': 'no-cache',
      'accept': 'application/json, text/plain, */*',
      'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36',
      'dnt': '1',
      'origin': 'https://www.emlakjet.com',
      'sec-fetch-site': 'same-site',
      'sec-fetch-mode': 'cors',
      'sec-fetch-dest': 'empty',
      'referer': 'https://www.emlakjet.com/satilik-konut/istanbul/',
      'accept-language': 'en,tr-TR;q=0.9,tr;q=0.8'
    }

    response = do_request(requests.request,method="GET",url=url, headers=headers, data = payload)
    return Dict(json.loads(response.text))


In [ ]:
def get_il_demography(CityId):
    if not isinstance(CityId,str):
        CityId=str(CityId)
    url = "https://app.endeksa.com/demography"

    payload = "{\"Level\":1,\"CityId\":"+CityId+"}"
    headers = {
      'authority': 'app.endeksa.com',
      'pragma': 'no-cache',
      'cache-control': 'no-cache',
      'accept': 'application/json, text/plain, */*',
      'dnt': '1',
      'accept-language': 'tr',
      'user-agent': '4fe54daf-c9bd-441f-8e98-24e9d480dc33',
      'content-type': 'application/json;charset=UTF-8',
      'origin': 'https://www.endeksa.com',
      'referer': 'https://www.endeksa.com/',
    }

    response = do_request(requests.request,method="POST",url = url, headers=headers, data = payload)
    return Dict(json.loads(response.text))

In [ ]:
#--Isimize yarayacak kolonlari donusturelim

selected_dicts={
    "CityId":"EMLAKJET_IL_KOD",
    "PopulationTotal":"TOPLAM NUFUS",
    "PopulationDensity":"KM2 DUSEN INSAN",
    "PopulationMale":"ERKEK NUFUS",
    "PopulationMaleRatio":"ERKEK NUFUS ORAN",
    "PopulationFemale":"KADIN NUFUS",
    "PopulationFemaleRatio":"KADIN NUFUS ORAN",
    "PopulationFemale":"KADIN NUFUS",
    "PopulationFemaleRatio":"KADIN NUFUS ORAN",
    "PopulationYoung":"GENÇ NUFUS",
    "PopulationYoungRatio":"GENC NUFUS ORAN",
    "PopulationMiddle":"ORTA YAS NUFUS",
    "PopulationMiddleRatio":"ORTA YAS NUFUS ORAN",
    "PopulationElder":"YASLI NUFUS",
    "PopulationElderRatio":"YASLI NUFUS ORAN",
    "Age_0_4_Total":"TOPLAM YAS 0-4",
    "Age_0_4_Male":"ERKEK YAS 0-4",
    "Age_0_4_Female":"KADIN YAS 0-4",
    "Age_5_9_Total":"TOPLAM YAS 5-9",
    "Age_5_9_Male":"ERKEK YAS 5-9",
    "Age_5_9_Female":"KADIN YAS 5-9",
    "Age_10_14_Total":"TOPLAM YAS 10-14",
    "Age_10_14_Male":"ERKEK YAS 10-14",
    "Age_10_14_Female":"KADIN YAS 10-14",
    "Age_15_19_Total":"TOPLAM YAS 15-19",
    "Age_15_19_Male":"ERKEK YAS 15-19",
    "Age_15_19_Female":"KADIN YAS 15-19",
    "Age_20_24_Total":"TOPLAM YAS 20-24",
    "Age_20_24_Male":"ERKEK YAS 20-24",
    "Age_20_24_Female":"KADIN YAS 20-24",
    "Age_25_29_Total":"TOPLAM YAS 25-29",
    "Age_25_29_Male":"ERKEK YAS 25-29",
    "Age_25_29_Female":"KADIN YAS 25-29",
    "Age_30_34_Total":"TOPLAM YAS 30-34",
    "Age_30_34_Male":"ERKEK YAS 30-34",
    "Age_30_34_Female":"KADIN YAS 30-34",
    "Age_35_39_Total":"TOPLAM YAS 35-39",
    "Age_35_39_Male":"ERKEK YAS 35-39",
    "Age_35_39_Female":"KADIN YAS 35-39",
    "Age_40_44_Total":"TOPLAM YAS 40-44",
    "Age_40_44_Male":"ERKEK YAS 40-44",
    "Age_40_44_Female":"KADIN YAS 40-44",
    "Age_45_49_Total":"TOPLAM YAS 45-49",
    "Age_45_49_Male":"ERKEK YAS 45-49",
    "Age_45_49_Female":"KADIN YAS 45-49",
    "Age_50_54_Total":"TOPLAM YAS 50-54",
    "Age_50_54_Male":"ERKEK YAS 50-54",
    "Age_50_54_Female":"KADIN YAS 50-54",
    "Age_55_59_Total":"TOPLAM YAS 55-59",
    "Age_55_59_Male":"ERKEK YAS 55-59",
    "Age_55_59_Female":"KADIN YAS 55-59",
    "Age_60_64_Total":"TOPLAM YAS 60-64",
    "Age_60_64_Male":"ERKEK YAS 60-64",
    "Age_60_64_Female":"KADIN YAS 60-64",
    "Age_65_Total":"TOPLAM YAS 65+",
    "Age_65_Male":"ERKEK YAS 65+",
    "Age_65_Female":"KADIN YAS 65+",
    "MarriedNever":"BEKAR",
    "Married":"EVLI",
    "Divorced":"BOSANMIS",
    "Widow":"DUL",
    "HousingCount":"KONUT ADET",
    "SummerResortCount":"YAZLIK ADET",
    "CommercialCount":"ISYERI ADET",
    "OwnerShare":"EV SAHIBI ORAN",
    "RentedShare":"KIRACI ORAN",
    "EduUnknown":"EGITIM BILINMIYOR",
    "EduNonLiterated":"OKURYAZAR DEGIL",
    "EduLiteratedUntutored":"OKURYAZAR",
    "EduPrimarySchool":"ILKOKUL",
    "EduMiddleSchool":"ORTAOKUL",
    "EduPrimaryEducation":"ILKOGRETIM",
    "EduHighSchool":"LISE",
    "EduLicenseDegree":"LISANS",
    "EduGraduate":"YUKSEK LISANS",
    "EduDoctorate":"DOKTORA",
    "EducationTotal":"TOPLAM EGITIM NUFUS",
    "SavingTotal":"KISI BASI ORTALAMA TASARRUF",
    "ExpenseTotal":"KISI BASI ORTALAMA HARCAMA",
    "ExpenseFood":"KISI BASI ORTALAMA GIDA HARCAMA",
    "ExpenseAlcoholAndSmoking":"KISI BASI ORTALAMA ALKOL-SIGARA HARCAMA",
    "ExpenseClothing":"KISI BASI ORTALAMA KIYAFET HARCAMA",
    "ExpenseShelter":"KISI BASI ORTALAMA KIRA HARCAMA",
    "ExpenseFurniture":"KISI BASI ORTALAMA MOBILYA HARCAMA",
    "ExpenseHealth":"KISI BASI ORTALAMA SAGLIK HARCAMA",
    "ExpenseTransportation":"KISI BASI ORTALAMA ULASIM HARCAMA",
    "ExpenseCommunication":"KISI BASI ORTALAMA ILETISIM HARCAMA",
    "ExpenseEntertainment":"KISI BASI ORTALAMA EGLENCE HARCAMA",
    "ExpenseEducation":"KISI BASI ORTALAMA EGITIM HARCAMA",
    "ExpenseRestaurant":"KISI BASI ORTALAMA RESTAURANT HARCAMA",
    "ExpenseOther":"KISI BASI ORTALAMA DIGER HARCAMA",
    "SesGroupAPlusRatio":"SES A+ ORAN",
    "SesGroupARatio":"SES A ORAN",
    "SesGroupBRatio":"SES B ORAN",
    "SesGroupCRatio":"SES C ORAN",
    "SesGroupDRatio":"SES D ORAN",
    "SesGroupABRatio":"SES A+B ORAN",
    "SesGroupCDRatio":"SES C+D ORAN",
    "HouseholdCount":"TOPLAM HANE SAYISI",
    "HouseIncome":"HANE KISI BASI GELIR",
    "HouseIncomeTotal":"HANE TOPLAM GELIR",
    "CarRatio":"OZEL OTO SAHIPLIK ORAN",
    "CarCount":"TOPLAM OZEL OTO SAYISI",
    "LandPrice":"KONUT_M2 FIYATI",
    "AtmCount":"ATM SAYISI",
    "VehicleRatio":"TASIT ORAN",
    "VehicleCount":"TOPLAM TASIT SAYISI",
    "BankBranchCount":"BANKA SUBE SAYISI",
    "ECommerceDensity":"E-TICARET YAPAN NUFUS ORAN",
}

In [ ]:
#--IL Bazli veriyi cekelim

mydf= pd.DataFrame()
temp= pd.DataFrame()
for i in range(1,82):
    res= pd.DataFrame.from_dict(get_il_demography(i)["Demography"].to_dict(),orient="index").T
    res= res[list(selected_dicts.keys())].rename(columns=selected_dicts)
    #Sadece IL bazinda degerleri alacagim, amacim AKS-ILCE BOŞ veya  AKS-ILCE GEÇERSİZ olan degerlerde IL bazli verilerle IMPUTE.
    temp['EMLAKJET_IL_KOD'] = res['EMLAKJET_IL_KOD']
    temp['EMLAKJET_ILCE_KOD'] = (10000 - res['EMLAKJET_IL_KOD'])
    #-----
    temp['TOPLAM_NUFUS'] = res['TOPLAM NUFUS']
    temp['KM2_DUSEN_INSAN'] = res['KM2 DUSEN INSAN']
    temp['ERKEK_NUFUS_ORAN'] = res['ERKEK NUFUS ORAN']
    temp['GENC_NUFUS_ORAN'] = res['GENC NUFUS ORAN']
    temp['ORTA_YAS_NUFUS_ORAN'] = res['ORTA YAS NUFUS ORAN']
    temp['YASLI_NUFUS_ORAN'] = res['YASLI NUFUS ORAN']
    temp['YAS_0-19_ORAN']= (res['TOPLAM YAS 0-4'] + res['TOPLAM YAS 5-9'] + res['TOPLAM YAS 10-14'] + res['TOPLAM YAS 15-19']) / res['TOPLAM NUFUS']
    temp['YAS_20-24_ORAN'] = res['TOPLAM YAS 20-24'] / res['TOPLAM NUFUS']
    temp['YAS_25-29_ORAN'] = res['TOPLAM YAS 25-29'] / res['TOPLAM NUFUS']
    temp['YAS_30-34_ORAN'] = res['TOPLAM YAS 30-34'] / res['TOPLAM NUFUS']
    temp['YAS_35-39_ORAN'] = res['TOPLAM YAS 35-39'] / res['TOPLAM NUFUS']
    temp['YAS_40-44_ORAN'] = res['TOPLAM YAS 40-44'] / res['TOPLAM NUFUS']
    temp['YAS_45-49_ORAN'] = res['TOPLAM YAS 45-49'] / res['TOPLAM NUFUS']
    temp['YAS_50-54_ORAN'] = res['TOPLAM YAS 50-54'] / res['TOPLAM NUFUS']
    temp['YAS_55-59_ORAN'] = res['TOPLAM YAS 55-59'] / res['TOPLAM NUFUS']
    temp['YAS_60-64_ORAN'] = res['TOPLAM YAS 60-64'] / res['TOPLAM NUFUS']
    temp['YAS_65+_ORAN']   = res['TOPLAM YAS 65+']   / res['TOPLAM NUFUS']
    temp['ERKEK_YAS_0-19_ORAN']= (res['ERKEK YAS 0-4'] + res['ERKEK YAS 5-9'] + res['ERKEK YAS 10-14'] + res['ERKEK YAS 15-19']) / res['ERKEK NUFUS']
    temp['ERKEK_YAS_20-24_ORAN'] = res['ERKEK YAS 20-24'] / res['ERKEK NUFUS']
    temp['ERKEK_YAS_25-29_ORAN'] = res['ERKEK YAS 25-29'] / res['ERKEK NUFUS']
    temp['ERKEK_YAS_30-34_ORAN'] = res['ERKEK YAS 30-34'] / res['ERKEK NUFUS']
    temp['ERKEK_YAS_35-39_ORAN'] = res['ERKEK YAS 35-39'] / res['ERKEK NUFUS']
    temp['ERKEK_YAS_40-44_ORAN'] = res['ERKEK YAS 40-44'] / res['ERKEK NUFUS']
    temp['ERKEK_YAS_45-49_ORAN'] = res['ERKEK YAS 45-49'] / res['ERKEK NUFUS']
    temp['ERKEK_YAS_50-54_ORAN'] = res['ERKEK YAS 50-54'] / res['ERKEK NUFUS']
    temp['ERKEK_YAS_55-59_ORAN'] = res['ERKEK YAS 55-59'] / res['ERKEK NUFUS']
    temp['ERKEK_YAS_60-64_ORAN'] = res['ERKEK YAS 60-64'] / res['ERKEK NUFUS']
    temp['ERKEK_YAS_65+_ORAN']   = res['ERKEK YAS 65+']   / res['ERKEK NUFUS']
    temp['KADIN_YAS_0-19_ORAN']= (res['KADIN YAS 0-4'] + res['KADIN YAS 5-9'] + res['KADIN YAS 10-14'] + res['KADIN YAS 15-19']) / res['KADIN NUFUS']
    temp['KADIN_YAS_20-24_ORAN'] = res['KADIN YAS 20-24'] / res['KADIN NUFUS']
    temp['KADIN_YAS_25-29_ORAN'] = res['KADIN YAS 25-29'] / res['KADIN NUFUS']
    temp['KADIN_YAS_30-34_ORAN'] = res['KADIN YAS 30-34'] / res['KADIN NUFUS']
    temp['KADIN_YAS_35-39_ORAN'] = res['KADIN YAS 35-39'] / res['KADIN NUFUS']
    temp['KADIN_YAS_40-44_ORAN'] = res['KADIN YAS 40-44'] / res['KADIN NUFUS']
    temp['KADIN_YAS_45-49_ORAN'] = res['KADIN YAS 45-49'] / res['KADIN NUFUS']
    temp['KADIN_YAS_50-54_ORAN'] = res['KADIN YAS 50-54'] / res['KADIN NUFUS']
    temp['KADIN_YAS_55-59_ORAN'] = res['KADIN YAS 55-59'] / res['KADIN NUFUS']
    temp['KADIN_YAS_60-64_ORAN'] = res['KADIN YAS 60-64'] / res['KADIN NUFUS']
    temp['KADIN_YAS_65+_ORAN']   = res['KADIN YAS 65+']   / res['KADIN NUFUS']
    temp['BEKAR_ORAN']  = res['BEKAR'] / (res['BEKAR'] + res['EVLI'] + res['BOSANMIS'] + res['DUL'])
    temp['EVLI_ORAN']   = res['EVLI'] / (res['BEKAR'] + res['EVLI'] + res['BOSANMIS'] + res['DUL'])
    temp['BOSANMIS_ORAN'] = res['BOSANMIS'] / (res['BOSANMIS'] + res['EVLI'] + res['BOSANMIS'] + res['DUL'])
    temp['DUL_ORAN']    = res['DUL'] / (res['BEKAR'] + res['EVLI'] + res['BOSANMIS'] + res['DUL'])
    temp['KONUT_ORAN']  = res['KONUT ADET']  /  (res['KONUT ADET'] + res['YAZLIK ADET'] + res['ISYERI ADET'])
    temp['YAZLIK_ORAN'] = res['YAZLIK ADET'] /  (res['KONUT ADET'] + res['YAZLIK ADET'] + res['ISYERI ADET'])
    temp['ISYERI_ORAN'] = res['ISYERI ADET'] /  (res['KONUT ADET'] + res['YAZLIK ADET'] + res['ISYERI ADET'])
    #--EV SAHIPLIGI
    temp['EV_SAHIBI_ORAN'] = res['EV SAHIBI ORAN']
    temp['KIRACI_ORAN']    = res['KIRACI ORAN']
    #--EGITIM
    temp['EGITIM_YOK_ORAN']        = (res['EGITIM BILINMIYOR'] + res['OKURYAZAR DEGIL']) / res['TOPLAM EGITIM NUFUS']
    temp['EGITIM_ILKOKUL_ORAN']    = (res['OKURYAZAR'] +  res['ILKOKUL'] + res['ORTAOKUL'] + res['ILKOGRETIM']) / res['TOPLAM EGITIM NUFUS']
    temp['EGITIM_LISE_ORAN']       =  res['LISE'] / res['TOPLAM EGITIM NUFUS']
    temp['EGITIM_UNIVERSITE_ORAN'] = (res['LISANS'] + res['YUKSEK LISANS'] +  res['DOKTORA'])  / res['TOPLAM EGITIM NUFUS']
    #--HARCAMA
    temp["KISI_BASI_ORTALAMA_TASARRUF"] = res["KISI BASI ORTALAMA TASARRUF"]
    temp["KISI_BASI_ORTALAMA_HARCAMA"]  = res["KISI BASI ORTALAMA HARCAMA"]
    temp['KISI_BASI_ORT_GIDA_GIDER']         = res['KISI BASI ORTALAMA GIDA HARCAMA']         / res["KISI BASI ORTALAMA HARCAMA"]
    temp['KISI_BASI_ORT_ALKOL-SIGARA_GIDER'] = res['KISI BASI ORTALAMA ALKOL-SIGARA HARCAMA'] / res["KISI BASI ORTALAMA HARCAMA"]
    temp['KISI_BASI_ORT_KIYAFET_GIDER']      = res['KISI BASI ORTALAMA KIYAFET HARCAMA']      / res["KISI BASI ORTALAMA HARCAMA"]
    temp['KISI_BASI_ORT_KIRA_GIDER']         = res['KISI BASI ORTALAMA KIRA HARCAMA']         / res["KISI BASI ORTALAMA HARCAMA"]
    temp['KISI_BASI_ORT_MOBILYA_GIDER']      = res['KISI BASI ORTALAMA MOBILYA HARCAMA']      / res["KISI BASI ORTALAMA HARCAMA"]
    temp['KISI_BASI_ORT_SAGLIK_GIDER']       = res['KISI BASI ORTALAMA SAGLIK HARCAMA']       / res["KISI BASI ORTALAMA HARCAMA"]
    temp['KISI_BASI_ORT_ULASIM_GIDER']       = res['KISI BASI ORTALAMA ULASIM HARCAMA']       / res["KISI BASI ORTALAMA HARCAMA"]
    temp['KISI_BASI_ORT_ILETISIM_GIDER']     = res['KISI BASI ORTALAMA ILETISIM HARCAMA']     / res["KISI BASI ORTALAMA HARCAMA"]
    temp['KISI_BASI_ORT_EGLENCE_GIDER']      = res['KISI BASI ORTALAMA EGLENCE HARCAMA']      / res["KISI BASI ORTALAMA HARCAMA"]
    temp['KISI_BASI_ORT_EGITIM_GIDER']       = res['KISI BASI ORTALAMA EGITIM HARCAMA']       / res["KISI BASI ORTALAMA HARCAMA"]
    temp['KISI_BASI_ORT_RESTORAN_GIDER']     = res['KISI BASI ORTALAMA RESTAURANT HARCAMA']   / res["KISI BASI ORTALAMA HARCAMA"]
    #--SOSYOEKONOMİK STATÜ
    temp['SES_A+_ORAN']   = res['SES A+ ORAN']
    temp['SES_AveB_ORAN'] = res['SES A ORAN'] + res['SES B ORAN']
    temp['SES_CveD_ORAN'] = res['SES C ORAN'] + res['SES D ORAN']
    #--HANE GELIR
    temp['HANE_KISI_BASI_GELIR'] = res['HANE KISI BASI GELIR']
    temp['HANE_TOPLAM_GELIR'] = res['HANE TOPLAM GELIR']
    #--DIGER
    temp['OZEL_OTO_SAHIPLIK_ORAN'] = res['OZEL OTO SAHIPLIK ORAN']
    temp['KONUT_M2_FIYATI'] = res['KONUT_M2 FIYATI']
    temp['ATM_SAYISI'] = res['ATM SAYISI']
    temp['BANKA_SUBE_SAYISI'] = res['BANKA SUBE SAYISI']
    temp['E-TICARET_YAPAN_NUFUS_ORAN'] = res['E-TICARET YAPAN NUFUS ORAN']

    mydf= pd.concat((mydf, temp), ignore_index= True)
    
mydf.to_csv('/data/sipai/Cumhur/ENDEKSA_IL_BAZLI_DEMOGRAFIK.csv', sep = '~', encoding= 'ISO-8859-9'	
